# Loan Monthly Payment Calculation
In this notebook, we are proving results, and using our results to find the monthly payment we would have to make given an APY, an initial balance, and a number of months for the term of the loan. The following findings come from the general concepts found in https://www.investopedia.com/loan-calculator-5104934. I did not pull any equations from anywhere. These are all found from me messing around with the logic from the link.

## Imports

In [1]:
import numpy as np

## Calculate Current Principle

The general rule for calculating the next principle balance in the sequence is
$$
    P_n = P_{n-1}(1+\frac{r}{12}) - m_p = P_{n-1} + \frac{r}{12}P_{n-1} - m_p
    I_n = \frac{r}{12}P_{n}
$$
Where $m_p$ is the monthly payment, and $r$ is the APY interest rate. So,
$ P_n = P_{n-1} - (m_p - I_{n-1}) $, and $ Pr_n = m_p - I_{n-1} $ where $Pr_n$ is the monthly principle payment. You can see that $P_n = P_{n-1}(1+r/12) - m_p$ essentially subtracts the principle payment for that month.

If the initial principle is $P_0$, then $P_1 = P_0(1+\frac{r}{12}) - m_p$ and,
$$
    P_2 = P_1(1+\frac{r}{12}) - m_p \\
    = (P_0(1+\frac{r}{12}) - m_p)(1+\frac{r}{12}) - m_p\\
    = P_0(1+\frac{r}{12})^2 - m_p\Sigma^1_{k=0}(1+\frac{r}{12})^k
$$
More generally, $P_n = P_0(1+\frac{r}{12})^n -m_p\Sigma^{n-1}_{k=0}(1+\frac{r}{12})^k$.

The following function, `curprinciple`, uses the above result.

In [4]:
def curprinciple(p0, apy, y, mp):
    return p0*(1+apy/12)**y - mp*np.sum((1+apy/12)**np.arange(0,y))

## Calculate Cumulative Interest
By definition, the interest payment for month $n$ is $I_n = \frac{r}{12}P_n$, and so cumulative interest is,
$$
    \Sigma^n_{k=0}I_k = \frac{r}{12}\Sigma^n_{k=0}P_k.
$$
Using the definition of $P_n$ from before,
$$
    \Sigma^n_{k=0}I_k = \frac{r}{12}[P_0 + P_0\Sigma^n_{k=1}(1+\frac{r}{12})^n -m_p\Sigma^n_{k=1}\Sigma^{k-1}_{j=0}(1+\frac{r}{12})^j].
$$
If you think about $\Sigma^n_{k=1}\Sigma^{k-1}_{j=0}(1+\frac{r}{12})^j$ for a moment, you'll find,
$$
    \Sigma^n_{k=1}\Sigma^{k-1}_{j=0}(1+\frac{r}{12})^j = \Sigma^n_{k=0}(n-k)(1+\frac{r}{12})^k
$$
And $P_0 + P_0\Sigma^n_{k=1}(1+\frac{r}{12})^k =P_0\Sigma^n_{k=0}(1+\frac{r}{12})^n$

So,
$$
    \Sigma^n_{k=0}I_k = \frac{r}{12}[P_0\Sigma^n_{k=0}(1+\frac{r}{12})^n -m_p\Sigma^n_{k=0}(n-k)(1+\frac{r}{12})^k] = 
    \frac{r}{12}\Sigma^n_{k=0}(P_0-m_p(n-k))(1+\frac{r}{12})^k.
$$
We use this result in the function below.

In [5]:
def cumInterest2(p0, apy, y, mp):
    return apy*np.sum((p0 - mp*(y-np.arange(0,y+1)))*(1+apy/12)**np.arange(0,y+1))/12

## Calculate Monthly Payment for A Loan
It's not difficult from this point to find the monthly payment for a loan. As arguments, we know the starting balance, the APY, and the number of months we want to pay the loan $n$. Since $n$ is after the final payment, then $P_n = 0.$ By definition,
$$
    P_n = 0 = P_0(1+\frac{r}{12})^n -m_p\Sigma^{n-1}_{k=0}(1+\frac{r}{12})^k
$$

And by algebraic manipulation,
$$
    m_p = \frac{P_0(1+\frac{r}{12})^n}{\Sigma^{n-1}_{k=0}(1+\frac{r}{12})^k}
$$

The following function applies this same logic.

In [3]:
def findMp(p0, apy, y):
    return p0*(1+apy/12)**y/np.sum((1+apy/12)**np.arange(0,y))

The following result is obvious.

In [2]:
def cumPrinciplePayments(p0,apy,y,mp):
   return mp*y - cumInterest2(p0,apy,y,mp)

The following calculates cumulative interest using recursion.

In [6]:
def cumInterest1(p0, apy, y, mp):
    Iy = curprinciple(p0, apy, y, mp)*apy/12
    if y == 0:
        return Iy
    else:
        return Iy + cumInterest1(p0, apy, y-1, mp)

In [7]:
curprinciple(6009, 0.048, 4, 69) # Current principle at 4 months

5828.061983426304

In [19]:
# This calculates the monthly payment you need to pay the principle of 6009 with a 4.8% APY over 72 months
mp = findMp(6009, 0.048, 72) 
mp

96.21806114534229

In [20]:
# This shows that given this monthly payment we've calculated that it does indeed reduce the principle balance to 0 over
# 72 months
curprinciple(6009, 0.048, 72, mp)

0.0

In [10]:
# The cumulative interest paid over 72 months given the calculated monthly payment (mp)
cumInterest2(6009, 0.048, 72, mp)

918.700402464643

In [11]:
# Cumulative principle payments over 72 months for a given mp (which should be equivalent to the starting balance
# since this is the mp required to pay off the principle in 72 months.)
cumPrinciplePayments(6009, 0.048,72,mp)

6009.000000000002

In [27]:
mp = findMp(400000, 0.05, 30*12)
mp

2147.2864920485517

In [26]:
curprinciple(400000, 0.05, 342, mp)

37162.80073298304

In [12]:
cumInterest1(6009, 0.048, 72, mp)

918.700402464643